In [155]:
import numpy as np
import pandas as pd
import ast

In [157]:
credits=pd.read_csv("tmdb_5000_credits[1].csv")
movies=pd.read_csv("tmdb_5000_movies[1].csv")
credits.head(1)
movies.shape

(4803, 20)

In [158]:
# movies.head(1)
# I have two different data frame so I have to merge the dataframe so it would be easy and I can merge them by usinng their similar movie name or id
movies=movies.merge(credits,on="title")
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [161]:
movies.shape

(4809, 23)

In [163]:
# There are two many column I have to remove the unnecessary column from there,As we doing content based we have to find the tags 
# here the column which  are necessary for the  tags only kept all other are skip.
# here genre,id,keywords,title,overview,cast and crew aree important so I Make another new dataframe using them.
movies.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [165]:
movies=movies[['movie_id','title','keywords','overview','genres','cast','crew']]

In [167]:
movies.head(1)

,movie_id,title,keywords,overview,genres,cast,crew
0,19995,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [169]:
# Now I have to create a tags for this So I am going to combine/merge keywords,overview,genres,cast,crew
movies.isnull().sum()


movie_id    0
title       0
keywords    0
overview    3
genres      0
cast        0
crew        0
dtype: int64

In [171]:
# In overview there are 3 missing values So I have to remove or replace that value by NaN
# I prefer to delete cause there are only 3 and It will not affect any values.
movies.dropna(inplace=True)


In [173]:
movies.duplicated().sum()

0

In [175]:
# No duplicte So I can move to nect step;
# The format is dictionary list I have to convert that into list
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [177]:
# I need only name from this So I only extract the name for that create a helper function
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        # I have to used ast because there is string and I can't travserse in list so I have to make in integers,list
        L.append(i['name'])
    return L


In [179]:
movies['genres']=movies['genres'].apply(convert)

In [180]:
# movies['keywords']

In [183]:
movies['keywords']=movies['keywords'].apply(convert)

In [184]:
movies.head(1)

,movie_id,title,keywords,overview,genres,cast,crew
0,19995,Avatar,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [185]:
# for cast there are so many person I have to import any three from there so I have to extract the three i have to write the function fo rthe code 
def convert3(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter!=3:
            L.append(i['name'])
        else:
            break
    return L

In [186]:
movies['cast']=movies['cast'].apply(convert3)

In [190]:
# from crew also I need only directoer So I would extract director name only,
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
    return L


In [191]:
movies['crew']=movies['crew'].apply(fetch_director)

In [192]:
movies.head(1)

,movie_id,title,keywords,overview,genres,cast,crew
0,19995,Avatar,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]


In [193]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [194]:
movies.head(1)

,movie_id,title,keywords,overview,genres,cast,crew
0,19995,Avatar,"[culture clash, future, space war, space colon...","[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]


In [195]:
# there is Sam space then surname if another director has the same then it would be wrong so we have to clear space from there
# Transform of data 

In [196]:
movies['overview'] = movies['overview'].apply(lambda x:[i.replace(" ","") for i in x])

In [197]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

In [198]:
movies['tags']=movies['keywords']+movies['overview']+movies['genres']+movies['cast']+movies['crew']

In [199]:
# create a newdataframe where columns are movie_id,title and tags
new_df=movies[['movie_id','title','tags']]

In [200]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"[cultureclash, future, spacewar, spacecolony, ..."
1,285,Pirates of the Caribbean: At World's End,"[ocean, drugabuse, exoticisland, eastindiatrad..."
2,206647,Spectre,"[spy, basedonnovel, secretagent, sequel, mi6, ..."
3,49026,The Dark Knight Rises,"[dccomics, crimefighter, terrorist, secretiden..."
4,49529,John Carter,"[basedonnovel, mars, medallion, spacetravel, p..."


In [201]:
# convert list to string 
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\KIIT\AppData\Local\Temp\ipykernel_17760\2779104012.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


In [202]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,cultureclash future spacewar spacecolony socie...
1,285,Pirates of the Caribbean: At World's End,ocean drugabuse exoticisland eastindiatradingc...
2,206647,Spectre,spy basedonnovel secretagent sequel mi6 britis...
3,49026,The Dark Knight Rises,dccomics crimefighter terrorist secretidentity...
4,49529,John Carter,basedonnovel mars medallion spacetravel prince...


In [203]:

# convert to lowercase
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

C:\Users\KIIT\AppData\Local\Temp\ipykernel_17760\113424304.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())


In [204]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,cultureclash future spacewar spacecolony socie...
1,285,Pirates of the Caribbean: At World's End,ocean drugabuse exoticisland eastindiatradingc...
2,206647,Spectre,spy basedonnovel secretagent sequel mi6 britis...
3,49026,The Dark Knight Rises,dccomics crimefighter terrorist secretidentity...
4,49529,John Carter,basedonnovel mars medallion spacetravel prince...


In [205]:
from sklearn.feature_extraction.text import CountVectorizer

In [206]:
cv=CountVectorizer(max_features=5000,stop_words="english")

In [209]:
vectors=cv.fit_transform(new_df["tags"]).toarray()
# we have to convert that cv inton array as the guven is in the sparse Matrixs and we have make in list of array 

In [210]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [211]:
# """when we cconvert in tags we have to find the similarity in the tags ,
# by matching words in the tags it would be wrong concept so we do vectorization where we convert each tags in vector mean 1d 
# they are represent in straight line in 2D if any of the line is close to each of them then we can say that the movie are closer 
# to each other,here stoping words such as and are is am are avoid and dont consists"""


In [212]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zoo', 'zooeydeschanel', 'zoëkravitz'],
      dtype=object)

In [213]:
# here are most similar words are repeated so I am going to stem the words 
# steam mean [loving,loved,love ] that would give as love only

In [214]:
import nltk

In [215]:
from nltk.stem.porter import PorterStemmer

In [216]:
ps=PorterStemmer()

In [217]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [218]:
new_df["tags"]=new_df["tags"].apply(stem)

C:\Users\KIIT\AppData\Local\Temp\ipykernel_17760\1027202188.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"]=new_df["tags"].apply(stem)


In [219]:
new_df

,movie_id,title,tags
0,19995,Avatar,cultureclash futur spacewar spacecoloni societ...
1,285,Pirates of the Caribbean: At World's End,ocean drugabus exoticisland eastindiatradingco...
2,206647,Spectre,spi basedonnovel secretag sequel mi6 britishse...
3,49026,The Dark Knight Rises,dccomic crimefight terrorist secretident burgl...
4,49529,John Carter,basedonnovel mar medallion spacetravel princes...
...,...,...,...
4804,9367,El Mariachi,unitedstates–mexicobarri leg arm paperknif gui...
4805,72766,Newlyweds,a newlyw couple' honeymoon is upend by the arr...
4806,231617,"Signed, Sealed, Delivered",date loveatfirstsight narrat investig team pos...
4807,126186,Shanghai Calling,when ambiti new york attorney sam is sent to s...


In [220]:
# Now I have to compute the distance and instead os doing the euclidean distance I am goin to take the coosine 
# distance It would be best for higher dimension

In [221]:
# I have to import the Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity

In [228]:
similarity=cosine_similarity(vectors)

In [230]:
similarity[0]

array([1.        , 0.07142857, 0.05216405, ..., 0.02326211, 0.02571722,
       0.        ])

In [232]:
# now i am gooing to compare the nearest value from to 1 to 0 but
# here the values are in order i have to get top 5 nearest so I will sort the value on the basis of index
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distance = similarity[movie_index]
    movie_list = sorted(list(enumerate(distance)), key=lambda x: x[1], reverse=True)[1:6]
    
    for i in movie_list:
        print(new_df.iloc[i[0]].title)

    

In [234]:
import pickle

In [236]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [238]:
pickle.dump(similarity,open("similarity.pkl",'wb'))